# 3_consensus_clustering.ipynb

This notebook performs consensus clustering on percolation participation matrices,
identifies critical nodes, and plots the top 15% consensus nodes on an MNI brain surface.

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.io
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.integrate import simpson
import matplotlib.pyplot as plt
from nilearn import plotting

In [ ]:
# === Set paths ===
data_dir = './data'
fig_dir = './figures/group_consensus'
os.makedirs(fig_dir, exist_ok=True)

In [ ]:
# === Load MNI coordinates (data/MNI_66_coords.txt) ===
coords_path = os.path.join(data_dir, 'MNI_66_coords.txt')
coords = pd.read_csv(coords_path, sep='	', header=None).values[:, :3]
n_nodes = coords.shape[0]

In [ ]:
# === Load percolation matrices ===
mat_paths = sorted([
    os.path.join(data_dir, f) for f in os.listdir(data_dir)
    if f.endswith('broadband_psi_adj_participation_in_percolation.mat')
])
consensus_votes = np.zeros(n_nodes)

In [ ]:
# === Process each subject ===
for path in mat_paths:
    mat = scipy.io.loadmat(path)
    if 'node_participation_at_percolation' not in mat:
        continue
    x = mat['node_participation_at_percolation'].T
    x_non_nan = ~np.isnan(np.sum(x, axis=0))
    d = np.diff(np.concatenate([[0], x_non_nan.astype(int), [0]]))
    starts, ends = np.where(d == 1)[0], np.where(d == -1)[0]
    if len(starts) == 0 or len(ends) == 0:
        continue
    longest = np.argmax(ends - starts)
    x_crop = x[:, starts[longest]:ends[longest]]
    x_crop = np.nan_to_num(x_crop)
    flattening = -np.linspace(1 / x_crop.shape[1], 1, x_crop.shape[1])
    weights = np.linspace(1, 1 / x_crop.shape[1], x_crop.shape[1])
    x_flat = x_crop + flattening[np.newaxis, :]
    x_weighted = x_flat * weights[np.newaxis, :]
    x_weighted = np.clip(x_weighted, 0, None)
    Z = linkage(x_weighted, method='ward')
    labels = fcluster(Z, 2, criterion='maxclust')
    auc_vals = np.array([simpson(x_weighted[i, :]) for i in range(x_weighted.shape[0])])
    crit_label = np.argmax([np.mean(auc_vals[labels == l]) for l in np.unique(labels)])
    crit_nodes = np.where(labels == crit_label + 1)[0]
    consensus_votes[crit_nodes] += 1

In [ ]:
# === Normalize consensus votes ===
n_subs = len(mat_paths)
consensus_score = consensus_votes / n_subs
def binary_top_15(score):
    threshold = np.percentile(score, 85)
    return (score >= threshold).astype(int)
crit_bin = binary_top_15(consensus_score)
highlight_vals = consensus_score * crit_bin

In [ ]:
# === Plot and save ===
plotting.plot_markers(
    node_values=highlight_vals,
    node_coords=coords,
    node_cmap='Reds',
    title='Percolation-Based Consensus Top Nodes',
    display_mode='lzr'
)
plt.savefig(os.path.join(fig_dir, 'percolation_consensus_top_nodes.png'), dpi=300)
plt.close()